# Logistic Regression with Neural Network mindset

just a feeling test on deeplearning

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy 
from PIL import Image
from scipy import ndimage
from lr_utils import load_dataset

In [8]:
train_set_x_orig,train_set_y,test_set_x_orig,test_set_y,classes=load_dataset()

In [9]:
index=25
plt.imshow(train_set_x_orig[index])
print ("y = " , str(train_set_y[:, index]) , ", it's a '" ,classes[np.squeeze(train_set_y[:, index])].decode("utf-8") ,  "' picture.")

y =  [1] , it's a ' cat ' picture.


### get the information of trainset

In [13]:
m_train=train_set_x_orig.shape[0]
m_test=test_set_x_orig.shape[0]
num_px=train_set_x_orig.shape[1]
num_px2=train_set_x_orig.shape[2]
print("number of training examples: m_train= ",m_train)
print("number of test examples: m_test= ",m_test)
print("height/width of each image: num_px= ",num_px)
print("sss",num_px2)

number of training examples: m_train=  209
number of test examples: m_test=  50
height/width of each image: num_px=  64
sss 64


In [31]:
# reshape them  np.array(train_set_x_orig)
#the first level stay stable, others flatern.
train_set_x_flatten=train_set_x_orig.reshape(train_set_x_orig.shape[0],-1).T
test_set_x_flatten=test_set_x_orig.reshape(test_set_x_orig.shape[0],-1).T
print("train_set_x_flattern shape: ",train_set_x_flatten.shape)
print("test_set_x_flattern shape: ",test_set_x_flattern.shape)
print("sanity check after reshape: ",train_set_x_flatten[0,0:10])

train_set_x_flattern shape:  (12288, 209)
test_set_x_flattern shape:  (12288, 50)
sanity check after reshape:  [ 17 196  82   1   9  84  56  19  63  23]


In [32]:
#standard the data
train_set_x=train_set_x_flatten/255.
test_set_x=test_set_x_flatten/255.

# building the parts of our algorithm

1. define the model structure
2. initialize model's parameters
3. Loop:
   calculate current loss(forward propagation)
   calculate current gradient(backward propagation)
   update parameters (gradient descent)

### helper functions

In [33]:
from basic_me import np_sigmoid
def sigmoid(z):
    """
    compute the sigmoid of z
    Arguments:
     z-- a scalar or numpy array of any size
     return :
     s--- sigmoid(z)
    """
    s=np_sigmoid(z)
    return s
    

In [34]:
print ("sigmoid([0, 2]) = " + str(sigmoid(np.array([0,2]))))

sigmoid([0, 2]) = [ 0.5         0.88079708]


### 4.2 initialzing parameters

In [61]:
def initialize_with_zeros(dim):
    """
    w --0 vector and b--0
    Augument:
    dim -- size of w vector 
    
    returns :
    w -- vector 
    b -- scalar
    """
    #!!!attention ,can not use np.zeros(dim)
    w=np.zeros((dim,1))
    b=0
    assert(w.shape==(dim,1))
    assert(isinstance(b,float)or isinstance(b,int))
    return w,b
    
    

In [64]:
dim =2
w,b=initialize_with_zeros(dim)
ww=w.shape[0]
print(ww)
print("w= ",w)
print("b= ",b)

2
w=  [[ 0.]
 [ 0.]]
b=  0


### 4.3  forward and backward propagation

### this roi is weak

In [12]:
import numpy as np
xxx=np.array([[1,2,3]])
zzz=np.array([[4,5,6]])
print(xxx.T*zzz)
print(np.dot(xxx.T,zzz))
yyy=np.log(xxx)
1-xxx
print(np.log(xxx))

[[ 4  5  6]
 [ 8 10 12]
 [12 15 18]]
[[ 4  5  6]
 [ 8 10 12]
 [12 15 18]]
[[ 0.          0.69314718  1.09861229]]


You calculate the cost function: $J = -\frac{1}{m}\sum_{i=1}^{m}y^{(i)}\log(a^{(i)})+(1-y^{(i)})\log(1-a^{(i)})$

In [ ]:
from basic_me import np_sigmoid
def propagate(w,b,X,Y):
    """
    Arguments:
    w --- weights , a numpy array of size (num_px*num_px*3,1),wihch is equal to image pixels number
    b --- bias , a scalar
    X --- data of size (num_px*num_px*3, number of examples)
    Y --- true label of size( 1, number of examples )
    """
    #m --- number of examples
    m=X.shape[1]
    
    #A=w.T*X+b    my wrong and view the solution.....
    A=np_sigmoid(np.dot(w.T,X)+b)
    #Y*np.log(A)+(1-Y)np.log(1-A)  #mine is right , but i did not go on .......
    cost=-1/m*np.sum(Y*np.log(A)+(1-Y)*np.log(1-A))
    #be attention to use np.dot() but not A.T....
    #dw=1/m*X(A-Y).T
    dw=1/m*np.dot(X,(A-Y).T)
    db=1/m*np.sum(A-Y)
    
    assert(dw.shape==w.shape)
    assert(db.dtype==float)
    cost=np.squeeze(cost)
    assert(cost.shape==())
    grads={"dw":dw,
          "db":db}
    return grads,cost
    
                     
    

In [24]:
w, b, X, Y = np.array([[1],[2]]), 2, np.array([[1,2],[3,4]]), np.array([[1,0]])
grads, cost = propagate(w, b, X, Y)
print ("dw = " ,grads["dw"])
print ("db = " , grads["db"])
print ("cost = " , cost)

dw =  [[ 0.99993216]
 [ 1.99980262]]
db =  0.499935230625
cost =  6.00006477319


In [4]:
import numpy as np
temp1=np.array([[1,2,3],
               [3,4,5]])
temp2=np.array([3,4,5])
temp3=np.sum(temp1)
print(temp3)

18


### optimization

the update rule is $ \theta = \theta - \alpha \text{ }d\theta $  
and $ \alpha $ is the learning rate.

In [ ]:
def optimize(w,b,X,Y,num_iterations,learning_rate,print_cost=False):
    """
    this function optimizes w and b by runting a gradient descent.
    Arguments: 
    w---weights, a numpy of size(num_px*num_px*3,1)
    b---bias, a scalar
    X--- data of shape(num_px*num_px*3,number of examples)
    Y--- true "label" vector of shape(1, number of examples)
    num_iterations --- number of iterations of the optimization loop 
    learning_rate --- learning rate of the gradient descent update rule 
    print_cost --- true to print the loss every 100 steps
    
    returns :
    params -- dictionary containing the weights w and bias b
    grads -- dictionary containing the gradients of weights and bias with respect to the cost function
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve .
    """
    costs=[]
    for i in range(num_iterations):
        
    